In [1]:
# train_model.py
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# 数据加载
batch_size = 32
img_size = (224, 224)
dataset_path = "../dataset/Garbage classification/Garbage classification"

train_dataset = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

val_dataset = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

class_names = train_dataset.class_names
print(f"类别索引映射: {class_names}")

# 数据增强
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.4),
    layers.RandomZoom(0.4),
    layers.RandomContrast(0.3),
    layers.RandomBrightness(0.2),
    layers.RandomTranslation(0.2, 0.2),  # 平移增强
    layers.RandomShear(0.2)  # 形变增强
])

# 加载预训练模型
base_model = keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False  # 先冻结

# 构建模型
inputs = keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = keras.applications.mobilenet_v2.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.001))(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(len(class_names), activation="softmax")(x)
model = keras.Model(inputs, outputs)

# 计算类别权重
y_train = []
for images, labels in train_dataset:
    y_train.extend(labels.numpy())

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(class_weights))

# 编译模型
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# 早停策略
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True
)

# 训练模型
epochs = 20
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
    class_weight=class_weights,  # 这里才是正确的
    callbacks=[early_stopping]
)

# 解冻部分层进行微调
base_model.trainable = True
for layer in base_model.layers[:100]:   # 让前100层冻结
    layer.trainable = False

# 重新编译模型
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.00001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# 继续训练（微调）
fine_tune_epochs = 10
history_fine = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=fine_tune_epochs,
    callbacks=[early_stopping]
)

# 保存模型
model.save("classification_model.keras")
print("模型已保存！")

Found 2527 files belonging to 6 classes.
Using 2022 files for training.
Found 2527 files belonging to 6 classes.
Using 505 files for validation.
类别索引映射: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
Epoch 1/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 51s 713ms/step - accuracy: 0.2838 - loss: 2.0990 - val_accuracy: 0.6891 - val_loss: 1.1562
Epoch 2/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 44s 679ms/step - accuracy: 0.5606 - loss: 1.3517 - val_accuracy: 0.7604 - val_loss: 0.9184
Epoch 3/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 43s 677ms/step - accuracy: 0.6395 - loss: 1.1917 - val_accuracy: 0.7842 - val_loss: 0.8371
Epoch 4/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 43s 678ms/step - accuracy: 0.6582 - loss: 1.1190 - val_accuracy: 0.7921 - val_loss: 0.8350
Epoch 5/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 44s 688ms/step - accuracy: 0.6878 - loss: 1.0839 - val_accuracy: 0.7980 - val_loss: 0.8250
Epoch 6/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 44s 691ms/step - accuracy: 0.6834 - loss: 1.0376 - val_accuracy: 0.7980 - val_loss: 0.7932
Epoch 7/20
6